# Project

### Goal: Improve our original estimate of the log error by using clustering methodologies.

## Acquisition, Prep, and Initial Exploration

Using the notebook and files you created during the exercises make any changes, additions, etc. you want at this point. NOTE: You will NOT be splitting into train and test at this point.

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Impute missing values
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format
pd.set_option('display.max_columns', None, 'display.max_rows', None)

import acquire
import wrangle_zillow

In [2]:
zillow = acquire.get_zillow(acquire.sql)
# Save file to csv so I can quit asking the server for this data
zillow.to_csv('zillow.csv')

In [3]:
zillow = pd.read_csv('zillow.csv', index_col='id')
# drop extra column that comes in from csv files
# zillow = zillow.drop(columns='Unnamed: 0')

In [4]:
orig_rows = zillow.shape[0]
orig_cols = zillow.shape[1]
print (f'There are {orig_rows} rows and {orig_cols} columns in the starting dataframe.')

There are 77575 rows and 67 columns in the starting dataframe.


In [5]:
# Drop rows:
# Restrict df to only properties that meet single use criteria
single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
zillow = zillow[zillow.propertylandusetypeid.isin(single_use)]

In [6]:
# Restrict df to only those properties with at least 1 bath & bed
zillow = zillow[(zillow.bedroomcnt > 0) & (zillow.bathroomcnt > 0)]

In [7]:
print (f'There are now {zillow.shape[0]} rows in the zillow dataframe.')

There are now 71406 rows in the zillow dataframe.


In [8]:
# Change the Y in taxdelinquencyflag to 1
zillow.taxdelinquencyflag = np.where(zillow.taxdelinquencyflag == 'Y', 1, 0)
zillow.taxdelinquencyflag.value_counts()

0    68807
1     2599
Name: taxdelinquencyflag, dtype: int64

In [9]:
# Add column for counties
zillow['county'] = np.where(zillow.fips == 6037, 'Los_Angeles',
                            np.where(zillow.fips == 6059, 'Orange', 'Ventura'))

In [10]:
# Address that zip code with too many digits
zillow[zillow.regionidzip > 100000].regionidzip.value_counts()

399,675.00    12
Name: regionidzip, dtype: int64

In [11]:
# pretty sure its just an extra digit
# check if 99675 is a zipcode: nope, so its probably a typo
zillow[zillow.regionidzip == 99675].regionidzip.value_counts()

Series([], Name: regionidzip, dtype: int64)

In [12]:
zillow['regionidzip'] = zillow['regionidzip'].replace(399675, 99675)

### Ideas:

1. Data types:

Write a function that takes in a dataframe and a list of column names and returns the dataframe with the datatypes of those columns changed to a non-numeric type.
Use this function to appropriately transform any numeric columns that should not be treated as numbers.

In [13]:
def numeric_to_object(df, num_cols):
    """
    Takes in a dataframe and a list of the columns to be transformed. 
    Changes the type of each column in the list to object type.
    """
    for col in num_cols:
        df[col] = df[col].astype('object')
    return df

In [14]:
# added this function to be sure numeric columns that should be integers and not floats are treated as such
def numeric_to_int(df, num_cols):
    """
    Takes in a dataframe and a list of the columns to be transformed. 
    Changes the type of each column in the list to integer type.
    """
    for col in num_cols:
        df[col] = df[col].astype('int')
    return df

Write a function that accepts the zillow data frame and returns the data frame with the missing values filled in.

In [16]:
def fill_nulls_with_zero(df, col_names):
    for col in col_names:
        df[col] = df[col].fillna(0)
    return df

In [17]:
# Test function
null_cols = ['airconditioningtypeid', 'basementsqft', 'decktypeid',
             'fireplacecnt', 'garagecarcnt', 'garagetotalsqft',
             'hashottuborspa', 'lotsizesquarefeet', 'poolcnt',
             'poolsizesum', 'taxdelinquencyyear']

zillow = fill_nulls_with_zero(zillow, null_cols)

3. Missing Values: Of the remaining missing values, can they be imputed or otherwise estimated?

Impute those that can be imputed with the method you feel best fits the attribute.

In [18]:
# Heatingorsystemtypeid - None type is 13
# Assume number of stories is 1
# Assume unitcnt is 1

def handle_other_nulls(df):
    df.heatingorsystemtypeid.fillna(13, inplace=True)
    df.numberofstories.fillna(1, inplace=True)
    df.unitcnt.fillna(1, inplace=True)
    return df

In [19]:
# Test function

zillow = handle_other_nulls(zillow)

In [20]:
# use scikit learn's IterativeImputer
# pre-emptively drop unnecessary columns

zillow = wrangle_zillow.remove_columns(zillow, 
                                       ['parcelid',
                                        'architecturalstyletypeid',
                                        'buildingclasstypeid',
                                        'finishedsquarefeet13',
                                        'finishedsquarefeet15',
                                        'finishedsquarefeet50',
                                        'finishedsquarefeet6',
                                        'finishedfloor1squarefeet',
                                        'pooltypeid10',
                                        'pooltypeid2',
                                        'pooltypeid7',
                                        'fireplaceflag',
                                        'airconditioningdesc',
                                        'storydesc',
                                        'heatingorsystemdesc',
                                        'architecturalstyledesc',
                                        'buildingclassdesc',
                                        'typeconstructiondesc',
                                        'yardbuildingsqft17',
                                        'yardbuildingsqft26',
                                        'calculatedbathnbr',
                                        'fullbathcnt',
                                        'threequarterbathnbr',
                                        'typeconstructiontypeid',
                                        'storytypeid',
                                        'propertyzoningdesc', 
                                        'calculatedfinishedsquarefeet', 
                                        'regionidneighborhood',
                                        'regionidcity',
                                        'regionidcounty',
                                        'propertylandusetypeid',
                                        'rawcensustractandblock',
                                        'propertylandusedesc',
                                        'assessmentyear'])

In [21]:
zillow.isna().sum().sort_values(ascending = False)

buildingqualitytypeid         26529
censustractandblock             213
finishedsquarefeet12            177
structuretaxvaluedollarcnt       79
yearbuilt                        42
regionidzip                      40
taxamount                         5
landtaxvaluedollarcnt             1
taxvaluedollarcnt                 1
garagetotalsqft                   0
garagecarcnt                      0
fireplacecnt                      0
fips                              0
county                            0
decktypeid                        0
heatingorsystemtypeid             0
bedroomcnt                        0
bathroomcnt                       0
basementsqft                      0
hashottuborspa                    0
poolcnt                           0
latitude                          0
longitude                         0
lotsizesquarefeet                 0
transactiondate                   0
poolsizesum                       0
propertycountylandusecode         0
roomcnt                     

In [22]:
# Try using imputer function here
# imputer function requires only numerical columns

num_vars = list(zillow.select_dtypes('number').columns)
null_df = zillow[num_vars]

imp = IterativeImputer(random_state=423)

imp.fit(null_df)
imputed_vals = pd.DataFrame(data=imp.transform(null_df),
                            columns=num_vars)
imputed_vals.head()

,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,decktypeid,finishedsquarefeet12,fips,fireplacecnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,regionidzip,roomcnt,unitcnt,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror
0,0.00,0.00,2.00,3.00,8.00,0.00,"2,107.00","6,037.00",0.00,0.00,0.00,0.00,2.00,"34,222,559.00","-118,617,387.00","9,158.00",1.00,0.00,"96,339.00",0.00,1.00,"1,972.00",1.00,"249,655.00","624,139.00","374,484.00","7,659.36",0.00,0.00,"60,371,132,321,007.00",-0.01
1,1.00,0.00,2.00,4.00,8.00,0.00,"1,882.00","6,037.00",0.00,0.00,0.00,0.00,2.00,"34,220,261.00","-118,616,409.00","9,035.00",1.00,0.00,"96,339.00",0.00,1.00,"1,972.00",1.00,"253,000.00","660,000.00","407,000.00","8,123.91",0.00,0.00,"60,371,132,321,007.00",0.02
2,1.00,0.00,2.00,4.00,8.00,0.00,"1,882.00","6,037.00",0.00,0.00,0.00,0.00,2.00,"34,222,491.00","-118,616,854.00","9,800.00",0.00,0.00,"96,339.00",0.00,1.00,"1,972.00",1.00,"257,591.00","542,923.00","285,332.00","6,673.24",0.00,0.00,"60,371,132,321,007.00",0.08
3,0.00,0.00,2.00,3.00,8.00,0.00,"1,477.00","6,037.00",0.00,0.00,0.00,0.00,2.00,"34,221,864.00","-118,615,739.00","11,285.00",1.00,0.00,"96,339.00",0.00,1.00,"1,960.00",1.00,"57,968.00","78,031.00","20,063.00","1,116.46",0.00,0.00,"60,371,132,321,008.00",-0.04
4,0.00,0.00,2.00,4.00,8.00,0.00,"1,918.00","6,037.00",0.00,0.00,0.00,0.00,2.00,"34,220,619.00","-118,615,253.00","11,239.00",1.00,0.00,"96,339.00",0.00,1.00,"1,960.00",1.00,"167,869.00","415,459.00","247,590.00","5,239.85",0.00,0.00,"60,371,132,321,008.00",-0.01


In [23]:
# Build imputer function

def zillow_imputer(df):
    num_vars = list(df.select_dtypes('number').columns)
    imp = IterativeImputer(random_state=423)
    imp.fit(df[num_vars])
    df[num_vars] = imp.transform(df[num_vars])
    return df

In [24]:
# Test function 

zillow = zillow_imputer(zillow)
zillow.isna().sum().sort_values(ascending = False)

county                        0
fireplacecnt                  0
longitude                     0
latitude                      0
heatingorsystemtypeid         0
hashottuborspa                0
garagetotalsqft               0
garagecarcnt                  0
fips                          0
transactiondate               0
finishedsquarefeet12          0
decktypeid                    0
buildingqualitytypeid         0
bedroomcnt                    0
bathroomcnt                   0
basementsqft                  0
lotsizesquarefeet             0
poolcnt                       0
poolsizesum                   0
propertycountylandusecode     0
regionidzip                   0
roomcnt                       0
unitcnt                       0
yearbuilt                     0
numberofstories               0
structuretaxvaluedollarcnt    0
taxvaluedollarcnt             0
landtaxvaluedollarcnt         0
taxamount                     0
taxdelinquencyflag            0
taxdelinquencyyear            0
censustr

In [25]:
# Imputed values are floats - be sure to reset integers to int

# transform float cols to int
int_cols = ['basementsqft', 'bathroomcnt', 'bedroomcnt', 
            'finishedsquarefeet12', 'fireplacecnt',
            'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
            'lotsizesquarefeet', 'poolcnt', 'poolsizesum',
            'roomcnt', 'unitcnt', 'yearbuilt', 'numberofstories',
            'structuretaxvaluedollarcnt', 'taxvaluedollarcnt',
            'landtaxvaluedollarcnt', 'taxamount',
            'taxdelinquencyflag', 'taxdelinquencyyear']

zillow = numeric_to_int(zillow, int_cols)

# transform float cols to object
obj_cols = ['regionidzip', 'airconditioningtypeid',
            'buildingqualitytypeid', 'decktypeid',
            'heatingorsystemtypeid', 'censustractandblock']

zillow = numeric_to_object(zillow, obj_cols)

In [33]:
zillow.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71406 entries, 1087254 to 2758757
Data columns (total 34 columns):
airconditioningtypeid         71406 non-null object
basementsqft                  71406 non-null int64
bathroomcnt                   71406 non-null int64
bedroomcnt                    71406 non-null int64
buildingqualitytypeid         71406 non-null object
decktypeid                    71406 non-null object
finishedsquarefeet12          71406 non-null int64
fips                          71406 non-null float64
fireplacecnt                  71406 non-null int64
garagecarcnt                  71406 non-null int64
garagetotalsqft               71406 non-null int64
hashottuborspa                71406 non-null int64
heatingorsystemtypeid         71406 non-null object
latitude                      71406 non-null float64
longitude                     71406 non-null float64
lotsizesquarefeet             71406 non-null int64
poolcnt                       71406 non-null int64
poolsi

In [32]:
zillow.censustractandblock.describe()

count                71,406.00
unique               36,327.00
top      60,371,371,031,000.00
freq                     60.00
Name: censustractandblock, dtype: float64

In [27]:
zillow.censustractandblock.value_counts()

60,371,371,031,000.00     60
60,372,766,032,001.00     51
60,590,320,582,009.00     44
60,590,626,102,134.00     41
60,371,393,022,000.00     40
60,590,626,225,003.00     40
60,375,766,011,005.00     39
60,379,201,162,006.00     37
60,379,200,312,000.00     37
60,379,203,391,054.00     36
60,590,524,272,002.00     34
60,375,776,041,024.00     33
60,372,964,011,008.00     31
60,379,203,301,005.00     30
60,590,524,202,001.00     29
60,373,004,001,007.00     29
60,375,760,011,000.00     29
60,590,741,071,005.00     28
60,590,320,581,007.00     28
60,590,524,173,014.00     28
61,110,072,021,000.00     28
60,371,395,031,002.00     27
60,375,766,021,002.00     27
61,110,074,023,000.00     26
60,372,079,002,009.00     26
60,371,396,001,000.00     25
60,590,863,031,016.00     25
60,590,219,182,001.00     25
60,590,626,481,001.00     25
60,590,013,032,001.00     25
60,372,766,031,001.00     24
60,371,417,002,000.00     24
60,590,626,402,000.00     24
60,590,320,571,003.00     24
61,110,075,073

In [34]:
plt.figure(figsize=(14,8))
sns.scatterplot(x='longitude', y='latitude', hue='censustractandblock', data=zillow)

AttributeError: 'float' object has no attribute 'view'

<Figure size 1008x576 with 0 Axes>

Decide whether to remove the rows or columns of any that cannot be reasonably imputed.

Document your reasons for the decisions on how to handle each of those.

4. Outliers: Original from exercises. Adapt as you see fit.

Write a function that accepts a series (i.e. one column from a data frame) and summarizes how many outliers are in the series. This function should accept a second parameter that determines how outliers are detected, with the ability to detect outliers in 3 ways: IQR, standard deviations (z-score), percentiles)

Use your function defined above to identify columns where you should handle the outliers.

Write a function that accepts the zillow data frame and removes the outliers. You should make a decision and document how you will remove outliers.

Is there erroneous data you have found that you need to remove or repair? If so, take action.

Are there outliers you want to "squeeze in" to a max value? (e.g. all bathrooms > 6 => bathrooms = 6). If so, make those changes.

## Exploration with Clustering

### Cluster the Target Variable

Perform clustering with logerror as the only feature used in the clustering algorithm. Decide on a number of clusters to use, and store the cluster predictions back onto your data frame as cluster_target. Look at the centroids that were produced in this process. What do they tell you?

Use the produced clusters to help you explore through visualization how logerror relates to other variables. (A common way to do this is to use color to indicate the cluster id, and the other variables can be your x-axis and y-axis. (hint: look at your swarmplot function)).

### Cluster Independent Variables

You should also perform some clustering based on a number of independent variables. Create and evaluate several clustering models based on subsets of the independent variables. Here are some ideas:

- Location, that is, latitude and longitude
- Size (finished square feet)
- Location and size

Be sure to use these new clusters in exploring your data, and interpret what these clusters tell you.

### Test the Significance of Clusters
Use statistical testing methods to determine whether the clusters you have created are significant in terms of their relationship to logerror.

## Modeling

### Feature Engineering

Remove variables that are not needed, wanted, useful, or are redundant.

Add any features you think may be useful.

In [ ]:
# Age = yearbuilt - 2017
# Tax Rate = 
# Avg tax rate by censustractandblockid
# $/sqft
# $/sqft structure only
# $/sqft lot only
# Avg $/sqft by censustractandblockid


Split your data into training and test sets.

In [ ]:
#train, test = wrangle_zillow.split_my_data(df, train_pct)

# what about scaling?

Create subsets of data if you would like to create multiple models and then merge (such as, a different model for each cluster or for each county).

## Model Selection

Train at least 3 different models (a model is different if there are changes in one or more of the following: features, hyper-parameters, algorithm). Create object, fit, predict & evaluate. Use mean absolute error or mean squared error to evaluate. Also, try regression algorithms you have not used before.

Evaluate your best model on your test data set to get an idea of your model's out of sample error.